In [ ]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/캡스톤/data/Anime3/'

In [ ]:
with open(path+"train_id_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open(path+"valid_id_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open(path+"test_id_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [ ]:
with open(path+"train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open(path+"valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open(path+"test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [ ]:
# ID
train_data[0] = tf.convert_to_tensor(train_data[0], dtype=tf.float32)
train_data[1] = tf.convert_to_tensor(train_data[1], dtype=tf.float32)

## Model

In [ ]:
!pip install keras_self_attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=0931476c6231d800dde2c4c4d1f63f59edd23acfde41ec6fd9de03f4eee1cc6f
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [ ]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten
from keras_self_attention import SeqSelfAttention
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 3
current_path ='data/'
local_path = 'data/'


def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

def avg_pool(X):
    return K.mean(X, axis = 3 , keepdims = True )

def max_pool(X):
    return K.max(X, axis = 3 , keepdims = True )

def con(avg_pool, max_pool):
    return Concatenate(axis=3)([avg_pool, max_pool])

def repeat_dim(X):
    return K.repeat_elements(X, 1945, 2 )

def repeat_dim_2(X):
    return K.repeat_elements(X, 50, 3  )

def val_repeat_dim(X):
    return K.repeat_elements(X, 1945, 1 )

def val_repeat_dim_2(X):
    return K.repeat_elements(X, 50, 2 )

def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [ ]:
# set parameters:
# V = 142381
# V = 226928
V = 48456 #33531 # len(tokenizer.word_index)
embedding_dim = 50
max_len = 1   # 1945  # maxlen
filters_num = 64
kernel_size = 3

In [ ]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))
input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
self_att = SeqSelfAttention(attention_activation='sigmoid')
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu' ,
                padding = 'same')

maxpool1d = MaxPool1D(max_len  )

dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(conv1d)(input_vector)
validation_conv = conv1d(validation_vector)

selfatt_vector = TimeDistributed(self_att)(convolutional_vector)
validation_selfatt = self_att(validation_conv)

maxpooling_vector = TimeDistributed(maxpool1d)(selfatt_vector)
validation_maxpooling = maxpool1d(validation_selfatt)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [ ]:

print(input_vector.shape, validation_vector.shape,
  convolutional_vector.shape ,  validation_conv.shape ,  
  selfatt_vector.shape ,  validation_selfatt.shape, 
  maxpooling_vector.shape , validation_maxpooling.shape, 
  middle_output.shape , middle_validation.shape
     )

(None, 3, 1, 50) (None, 1, 50) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 15) (None, 1, 15)


In [ ]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output) #?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [ ]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', 'mape'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 3, 1, 50)    2422800     ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        2422800     ['input_2[0][0]']            

In [ ]:
'''hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 epochs=3,
                 verbose=10)
'''
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 15s 10ms/step - loss: 12.8201 - mse: 12.8201 - val_loss: 2.4453 - val_mse: 2.4453
Epoch 2/30
256/256 [==============================] - 2s 8ms/step - loss: 2.0887 - mse: 2.0887 - val_loss: 2.3813 - val_mse: 2.3813
Epoch 3/30
256/256 [==============================] - 2s 8ms/step - loss: 2.0238 - mse: 2.0238 - val_loss: 2.4216 - val_mse: 2.4216
Epoch 4/30
256/256 [==============================] - 2s 8ms/step - loss: 1.9928 - mse: 1.9928 - val_loss: 2.4060 - val_mse: 2.4060
Epoch 5/30
256/256 [==============================] - 2s 8ms/step - loss: 1.9748 - mse: 1.9748 - val_loss: 2.4136 - val_mse: 2.4136
Epoch 6/30
256/256 [==============================] - 2s 8ms/step - loss: 1.9612 - mse: 1.9612 - val_loss: 2.4297 - val_mse: 2.4297
Epoch 7/30
256/256 [==============================] - 2s 8ms/step - loss: 1.9527 - mse: 1.9527 - val_loss: 2.3884 - val_mse: 2.3884
Epoch 8/30
256/256 [==============================] - 2s 8ms/step - loss

In [ ]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
# test_number = test.shape[0]
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2465/2500 [============================>.] - ETA: 0s - loss: 2.6816 - mse: 2.6816

In [ ]:
# model.save("/content/drive/MyDrive/Colab Notebooks/캡스톤/IMDB_ADLFM_SA")